In [7]:
import ctypes
import time

SendInput = ctypes.windll.user32.SendInput


UP = 0xC8 
DOWN = 0xD0
S = 0x1F
D = 0x20

NP_2 = 0x50
NP_4 = 0x4B
NP_6 = 0x4D
NP_8 = 0x48

# C struct redefinitions 
PUL = ctypes.POINTER(ctypes.c_ulong)
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                 ("mi", MouseInput),
                 ("hi", HardwareInput)]

class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]

# Actuals Functions

def PressKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

def ReleaseKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008 | 0x0002, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

if __name__ == '__main__':
    PressKey(0x11)
    time.sleep(1)
    ReleaseKey(0x11)
    time.sleep(1)


In [8]:
import cv2
import numpy as np
import time
#from directkeys import PressKey, ReleaseKey, UP, DOWN
        

cap = cv2.VideoCapture(0)
time.sleep(5)
up = False
down = False
center = True


while(1):
    ret, img = cap.read()
    height, width, _ = img.shape
    blur =  cv2.blur(img,(3,3))
    hsv = cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)

    lower = np.array([0,48,80])
    upper = np.array([20,255,255])
    mask = cv2.inRange(hsv,lower, upper)

    kernel_square = np.ones((11,11),np.uint8)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
        
    #Perform morphological transformations to filter out the background noise
    #Dilation increase skin color area
    #Erosion increase skin color area
    dilation = cv2.dilate(mask,kernel_ellipse,iterations = 1)
    erosion = cv2.erode(dilation,kernel_square,iterations = 1)    
    dilation2 = cv2.dilate(erosion,kernel_ellipse,iterations = 1)    
    filtered = cv2.medianBlur(dilation2,5)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8))
    dilation2 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    kernel_ellipse= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilation3 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    median = cv2.medianBlur(dilation2,5)
    ret,thresh = cv2.threshold(median,127,255,0)
    _,contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, (122,122,0), 3)
    new = contours
    max_area=100
    ci=0
    if(len(contours) > 0):
        for i in range(len(contours)):
            cnt=contours[i]
            area = cv2.contourArea(cnt)
            if(area>max_area):
                max_area=area
                ci=i     
                #Largest area contour
        cnts = new[ci]
        moments = cv2.moments(cnts)
        
    #Central mass of first order moments
        if moments['m00']!=0:
            cx = int(moments['m10']/moments['m00']) # cx = M10/M00
            cy = int(moments['m01']/moments['m00']) # cy = M01/M00
        centerMass=(cx,cy)
        print(cx,cy)
        cv2.circle(img,centerMass,7,[100,0,255],2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img,'Center',tuple(centerMass),font,2,(0,0,0),2)
        if(cy < (height/2)-70 and center == True ):
            center = False
            up = True
            print('up')
            PressKey(UP)
            ReleaseKey(UP)
        if( cy > (height/2)-70 and cy < (height/2)+70 and (up == True or down == True)):
            center = True
            up = False
        if(cy > (height/2)+70 and center == True):
            center = False
            down = True
            print('down')
            PressKey(DOWN)
            ReleaseKey(DOWN)
    cv2.line(img,(0,int((height/2)-70)),(width,int((height/2)-70)),(255,255,255),5)
    cv2.line(img,(0,int((height/2)+70)),(width,int((height/2)+70)),(255,255,255),5)
    cv2.imshow('frame',img)
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break

cap.release()

cv2.destroyAllWindows()


496 259
496 259
497 260
496 263
497 262
494 261
495 260
496 264
497 263
497 264
498 262
532 14
up
530 27
567 58
484 267
497 272
496 274
499 273
488 272
496 276
499 276
495 276
497 279
495 275
495 275
499 274
538 44
up
498 278
490 274
494 279
491 275
578 68
up
463 282
468 264
494 259
510 248
211 301
226 276
219 255
219 243
214 238
211 234
206 238
202 239
198 243
194 245
192 240
198 246
199 244
200 248
207 251
209 248
210 247
212 247
215 249
215 247
214 246
212 245
212 248
212 249
213 245
215 245
216 249
213 247
215 246
217 249
217 249
217 245
219 246
219 247
220 246
224 264
221 266
219 266
224 265
214 259
217 247
217 249
219 250
225 253
226 267
230 267
232 265
532 262
530 255
537 257
540 257
540 256
537 257
543 264
538 258
536 259
541 258
611 241
540 260
541 259
536 274
535 260
534 260
542 262
535 257
535 262
532 260
598 242
523 258
524 256
524 255
533 258
529 264
536 262
537 267
536 261
539 263
515 253
547 262
547 261
542 255
556 260
232 265
231 256
236 256
238 257
239 253
236 248
237 